# Get Toronto Neighbourhood Data from Wikipedia

In [1]:
import pandas as pd
import numpy as np

In [2]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

In [3]:
df = tables[0]

In [4]:
df = df.loc[df['Borough'] != "Not assigned"]

In [5]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df[df['Postal Code'].duplicated(keep=False)]

,Postal Code,Borough,Neighbourhood


No duplicated postal codes

# Import Postal Co-ordinates

In [7]:
postal_coordinates = pd.read_csv('C:\\Users\\max\\Downloads\\FSA_Coordinates.csv')

In [8]:
postal_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df = df.merge(postal_coordinates, how='inner', on='Postal Code')

In [10]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Import Income Data

In [11]:
fsa_tax = pd.read_csv('C:\\Users\\max\\Downloads\\FSA_Tax_Payment.csv')

In [12]:
df = df.merge(fsa_tax, how='inner', on='Postal Code')
df.drop('Prov/Terr', inplace=True, axis=1)
df['Total Income per Cap'] = df['Total Income']/df['Total']
df['Net Income per Cap'] = df['Net Income']/df['Total']
df['Taxable Income per Cap'] = df['Taxable Income']/df['Total']

In [13]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Total,Total Income,Net Income,Taxable Income,Total Income per Cap,Net Income per Cap,Taxable Income per Cap
0,M3A,North York,Parkwoods,43.753259,-79.329656,26010,1.145870e+09,1.061663e+09,1.022821e+09,44054.978854,40817.493272,39324.144560
1,M4A,North York,Victoria Village,43.725882,-79.315572,11220,4.303120e+08,3.991580e+08,3.784060e+08,38352.228164,35575.579323,33726.024955
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,30460,1.538571e+09,1.423599e+09,1.348482e+09,50511.195010,46736.671044,44270.584373
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,15450,6.318830e+08,5.907090e+08,5.633980e+08,40898.576052,38233.592233,36465.889968
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN


# Examine Neighbourhoods

In [14]:
from geopy.geocoders import Nominatim
import folium
import requests

In [15]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

# Call to Four Square API

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(1304, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [21]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
...,...,...,...,...,...,...
"Willowdale, Willowdale East",30,30,30,30,30,30
"Willowdale, Willowdale West",6,6,6,6,6,6
Woburn,4,4,4,4,4,4


In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_onehot.shape

(1304, 230)

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038462,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0
90,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
91,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
92,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.166667,0.000000,0.0,0.0,0.0,0.0,0.0


In [26]:
toronto_grouped.shape

(94, 230)

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Deli / Bodega,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Sandwich Place,Pub,Yoga Studio,Dessert Shop,Dance Studio,Deli / Bodega,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Shopping Mall,Supermarket,Sushi Restaurant,Restaurant,Diner,Middle Eastern Restaurant,Deli / Bodega,Mobile Phone Shop
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Restaurant,Italian Restaurant,Grocery Store,Butcher,Pub,Fast Food Restaurant,Indian Restaurant,Sushi Restaurant


In [30]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [31]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1])

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

df_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Total,Total Income,Net Income,Taxable Income,Total Income per Cap,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,26010,1.145870e+09,1.061663e+09,1.022821e+09,44054.978854,...,Park,Food & Drink Shop,Yoga Studio,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,11220,4.303120e+08,3.991580e+08,3.784060e+08,38352.228164,...,Portuguese Restaurant,Coffee Shop,Hockey Arena,French Restaurant,Pizza Place,Donut Shop,Dog Run,Distribution Center,Discount Store,Curling Ice
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,30460,1.538571e+09,1.423599e+09,1.348482e+09,50511.195010,...,Coffee Shop,Park,Café,Bakery,Breakfast Spot,Theater,Yoga Studio,French Restaurant,Dessert Shop,Spa
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,15450,6.318830e+08,5.907090e+08,5.633980e+08,40898.576052,...,Clothing Store,Accessories Store,Arts & Crafts Store,Coffee Shop,Miscellaneous Shop,Boutique,Furniture / Home Store,Event Space,Vietnamese Restaurant,Women's Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,0.000000e+00,0.000000e+00,0.000000e+00,NaN,...,Coffee Shop,Diner,Burrito Place,Bank,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Distribution Center,Yoga Studio


In [33]:
null_columns = df_merged.columns[df_merged.isnull().any()]
null_rows = df_merged[df_merged.isnull().any(axis=1)][null_columns].index
df_merged.drop(df_merged.index[null_rows], inplace=True)

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighbourhood'], df_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
df_merged.loc[df_merged['Cluster Labels'] == 0, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Total,Total Income,Net Income,Taxable Income,Total Income per Cap,Net Income per Cap,Taxable Income per Cap,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,27040,7.925150e+08,745657000.0,691457000.0,29308.986686,27576.072485,25571.634615,0.0,Pizza Place,Playground,Cupcake Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
76,Etobicoke,26510,1.002759e+09,932733000.0,885284000.0,37825.688419,35184.194644,33394.341758,0.0,Mobile Phone Shop,Pizza Place,Sandwich Place,Bus Line,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
92,Etobicoke,16280,7.804080e+08,719650000.0,699815000.0,47936.609337,44204.545455,42986.179361,0.0,Pizza Place,Gym,Coffee Shop,Sandwich Place,Pub,Yoga Studio,Dessert Shop,Dance Studio,Deli / Bodega,Department Store


In [36]:
df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Total,Total Income,Net Income,Taxable Income,Total Income per Cap,Net Income per Cap,Taxable Income per Cap,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,11220,4.303120e+08,3.991580e+08,3.784060e+08,38352.228164,35575.579323,33726.024955,1.0,Portuguese Restaurant,Coffee Shop,Hockey Arena,French Restaurant,Pizza Place,Donut Shop,Dog Run,Distribution Center,Discount Store,Curling Ice
2,Downtown Toronto,30460,1.538571e+09,1.423599e+09,1.348482e+09,50511.195010,46736.671044,44270.584373,1.0,Coffee Shop,Park,Café,Bakery,Breakfast Spot,Theater,Yoga Studio,French Restaurant,Dessert Shop,Spa
3,North York,15450,6.318830e+08,5.907090e+08,5.633980e+08,40898.576052,38233.592233,36465.889968,1.0,Clothing Store,Accessories Store,Arts & Crafts Store,Coffee Shop,Miscellaneous Shop,Boutique,Furniture / Home Store,Event Space,Vietnamese Restaurant,Women's Store
6,Scarborough,51410,1.577233e+09,1.476645e+09,1.395635e+09,30679.498152,28722.913830,27147.150360,1.0,Fast Food Restaurant,Yoga Studio,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
7,North York,12110,1.508813e+09,1.399757e+09,1.371840e+09,124592.320396,115586.870355,113281.585467,1.0,Gym,Beer Store,Restaurant,Japanese Restaurant,Coffee Shop,Caribbean Restaurant,Dim Sum Restaurant,Discount Store,Chinese Restaurant,Sporting Goods Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Downtown Toronto,15670,6.032110e+08,5.655820e+08,5.293580e+08,38494.639438,36093.299298,33781.620932,1.0,Coffee Shop,Restaurant,Italian Restaurant,Café,Bakery,Park,Beer Store,Butcher,Pub,Caribbean Restaurant
96,Downtown Toronto,110,4.247400e+07,4.018700e+07,3.987900e+07,386127.272727,365336.363636,362536.363636,1.0,Café,Coffee Shop,Restaurant,Hotel,General Travel,Steakhouse,Pub,Bakery,Food Court,Pizza Place
97,Etobicoke,8770,1.210118e+09,1.119811e+09,1.089494e+09,137983.808438,127686.545040,124229.646522,1.0,River,Pool,Cupcake Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
98,Downtown Toronto,22510,1.139727e+09,1.056991e+09,1.031304e+09,50632.030209,46956.508219,45815.370946,1.0,Coffee Shop,Park,Beer Bar,Breakfast Spot,Bubble Tea Shop,Salon / Barbershop,Burger Joint,Restaurant,Ramen Restaurant,Pub


In [37]:
df_merged.loc[df_merged['Cluster Labels'] == 2, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Total,Total Income,Net Income,Taxable Income,Total Income per Cap,Net Income per Cap,Taxable Income per Cap,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,17270,5.299700e+08,500214000.0,470964000.0,30687.319050,28964.331210,27270.642733,2.0,Baseball Field,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
100,Etobicoke,16550,1.012287e+09,932074000.0,907459000.0,61165.377644,56318.670695,54831.359517,2.0,Baseball Field,Construction & Landscaping,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center


In [38]:
df_merged.loc[df_merged['Cluster Labels'] == 3, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Total,Total Income,Net Income,Taxable Income,Total Income per Cap,Net Income per Cap,Taxable Income per Cap,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,26010,1.145870e+09,1.061663e+09,1.022821e+09,44054.978854,40817.493272,39324.144560,3.0,Park,Food & Drink Shop,Yoga Studio,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
10,North York,22570,1.236338e+09,1.149592e+09,1.113588e+09,54777.935312,50934.514843,49339.299956,3.0,Park,Japanese Restaurant,Pub,Metro Station,Yoga Studio,Dance Studio,Donut Shop,Dog Run,Distribution Center,Discount Store
21,York,30200,1.046724e+09,9.854940e+08,9.355810e+08,34659.735099,32632.251656,30979.503311,3.0,Park,Women's Store,Pool,Yoga Studio,Curling Ice,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
35,East York,26970,1.366623e+09,1.246053e+09,1.205603e+09,50671.968854,46201.446051,44701.631442,3.0,Park,Convenience Store,Metro Station,Yoga Studio,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
49,North York,16150,5.492900e+08,5.151860e+08,4.853070e+08,34011.764706,31900.061920,30049.969040,3.0,Park,Construction & Landscaping,Bakery,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
52,North York,28490,1.147808e+09,1.065530e+09,1.025402e+09,40288.101088,37400.140400,35991.646192,3.0,Park,Yoga Studio,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
61,Central Toronto,12020,2.546182e+09,2.393008e+09,2.309339e+09,211828.785358,199085.524126,192124.708819,3.0,Park,Swim School,Bus Line,Yoga Studio,Dance Studio,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
64,York,19260,6.437130e+08,6.038850e+08,5.612060e+08,33422.274143,31354.361371,29138.421599,3.0,Park,Convenience Store,Yoga Studio,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
66,North York,6520,8.728170e+08,8.187290e+08,8.011730e+08,133867.638037,125571.932515,122879.294479,3.0,Park,Convenience Store,Yoga Studio,Dance Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
84,Scarborough,46190,1.301882e+09,1.211045e+09,1.130845e+09,28185.364798,26218.770297,24482.463737,3.0,Playground,Park,Yoga Studio,Curling Ice,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


In [41]:
df_merged.loc[df_merged['Cluster Labels'] == 3, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]].to_csv('C:\\Users\\max\\Desktop\\df.csv')